In [28]:
import time
import pySCION_initialise
import numpy as np
import scipy.io as sp
import matplotlib.pyplot as plt
from cmcrameri import cm
from scipy.interpolate import interp1d



In [ ]:
ocean_arcs = np.loadtxt('./data/ocean_arc_distribution.csv')
lat_bins = np.linspace(-90,90,int(185/5))
times = np.arange(0,1010, 10)
# -25 to 25 is our equatorial range
phanerozoic_tropical_arcs = ocean_arcs[15:23,0:61]
length_of_arcs_in_tropics = np.sum(phanerozoic_tropical_arcs, axis=0)

# flux with be width X length X fraction of the length that is arc
# scale by present-day flux of t/km2/a
# caclualte a base ocean arc flux
arc_cw_flux = 150 # t/km2/a
arc_width = 25 # km
arc_fraction = 0.5 # % of the length is arc rock
tropical_arc_weathering = length_of_arcs_in_tropics * arc_width * arc_fraction * arc_cw_flux
res = pySCION_initialise.pySCION_initialise(-1, tropical_arc_weathering)


In [ ]:
# store some results
arc_cw_fluxes = np.array([40, 150, 400])  # t/km2/a gaillardet ++ 2011
arc_widths = np.array([10, 25, 50])
arc_fractions = np.array([0.1, 0.25, 0.5])
temp_results = []
time_results = []
input_variables = []
arc_contributions_per_km2 = []
total_arc_contributions = []
for ind, arc_cw_flux in enumerate(arc_cw_fluxes):
    for ind2, width in enumerate(arc_widths):
        for ind3, fractions in enumerate(arc_fractions):
            tropical_arc_weathering = length_of_arcs_in_tropics * width * fractions * arc_cw_flux
            arc_contributions_per_km2.append(width * fractions * arc_cw_flux)
            total_arc_contributions.append(tropical_arc_weathering)
            res, interpstack, lipstack = pySCION_initialise.pySCION_initialise(
                -1, tropical_arc_weathering)
            
            temp_results.append(res.state.tempC)
            time_results.append(res.state.time_myr)
            input_variables.append((arc_cw_flux, width, fractions))

In [ ]:
# sorted_arc_contribution, sorted_temp_results, sorted_time_results, sorted_inputs = zip(
#    *sorted(zip(arc_contribution, temp_results, time_results, inputs)))

temp_results = np.asarray(temp_results, dtype=object)
time_results = np.asarray(time_results, dtype=object)
input_variables = np.asarray(input_variables)
arc_contributions_per_km2 = np.asarray(arc_contributions_per_km2)
total_arc_contributions = np.asarray(total_arc_contributions)


colors = cm.berlin(np.linspace(0, 1, len(arc_contributions_per_km2)))
sorted_inds = np.argsort(arc_contributions_per_km2)
plt.figure(figsize=(18,15))
for ind, sorted_ind in enumerate(sorted_inds):
    default_result = False
    label = f'weather flux {input_variables[sorted_ind][0]} || arc fraction {input_variables[sorted_ind][2]} || arc width {input_variables[sorted_ind][1]}'
    plt.subplot(211)
    plt.plot(time_results[sorted_ind], temp_results[sorted_ind],
             color=colors[ind], label=label, lw=2)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncols=1)
plt.xlim(-600, 0)
plt.ylim(12, 30)
plt.axvspan(-330, -260, alpha=0.25)
plt.axvspan(-39, 0, alpha=0.25)
plt.grid()
plt.xlabel('Time (Ma)')
plt.ylabel('Temp (°C)')

#plt.subplot(212)
for ind, sorted_ind in enumerate(sorted_inds):
    plt.subplot(212)
    plt.plot(times[:61]*-1, total_arc_contributions[sorted_ind],
             color=colors[ind], lw=2)
plt.xlim(-600, 0)
plt.semilogy()
plt.axvspan(-330, -260, alpha=0.25)
plt.axvspan(-39, 0, alpha=0.25)
plt.grid()
plt.xlabel('Time (Ma)')
plt.ylabel('CW flux (t/a)')   
plt.tight_layout()

In [ ]:
#nu uses the previous time step as well
arc_cw_fluxes = np.array([40, 150, 400])  # t/km2/a gaillardet ++ 2011
arc_widths = np.array([10, 25, 50])
arc_fractions = np.array([0.1, 0.25, 0.5])
nu_temp_results = []
nu_time_results = []
input_variables = []
nu_arc_contributions_per_km2 = []
nu_total_arc_contributions = []
for ind, arc_cw_flux in enumerate(arc_cw_fluxes):
    for ind2, width in enumerate(arc_widths):
        for ind3, fractions in enumerate(arc_fractions):

            tropical_arc_weathering = length_of_arcs_in_tropics * \
                width * fractions * arc_cw_flux
            # add previous step to current step
            nu_weather = []
            for ind, i in enumerate(tropical_arc_weathering):
                try:
                    nu_weather.append(i + tropical_arc_weathering[ind+1])
                except:
                    nu_weather.append(i)
            nu_weather = np.asarray(nu_weather)
            nu_arc_contributions_per_km2.append(width * fractions * arc_cw_flux)
            nu_total_arc_contributions.append(nu_weather)
            
            res, interpstack, lipstack = pySCION_initialise.pySCION_initialise(
                -1, nu_weather)

            nu_temp_results.append(res.state.tempC)
            nu_time_results.append(res.state.time_myr)
            input_variables.append((arc_cw_flux, width, fractions))

In [ ]:
# sorted_arc_contribution, sorted_temp_results, sorted_time_results, sorted_inputs = zip(
#    *sorted(zip(arc_contribution, temp_results, time_results, inputs)))

nu_temp_results = np.asarray(nu_temp_results, dtype=object)
nu_time_results = np.asarray(nu_time_results, dtype=object)
input_variables = np.asarray(input_variables)
nu_arc_contributions_per_km2 = np.asarray(nu_arc_contributions_per_km2)
nu_total_arc_contributions = np.asarray(nu_total_arc_contributions)


colors = cm.berlin(np.linspace(0, 1, len(nu_arc_contributions_per_km2)))
sorted_inds = np.argsort(nu_arc_contributions_per_km2)
plt.figure(figsize=(18, 15))
for ind, sorted_ind in enumerate(sorted_inds):
    default_result = False
    label = f'weather flux {input_variables[sorted_ind][0]} || arc fraction {input_variables[sorted_ind][2]} || arc width {input_variables[sorted_ind][1]}'
    plt.subplot(211)
    plt.plot(nu_time_results[sorted_ind], nu_temp_results[sorted_ind],
             color=colors[ind], label=label, lw=2)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncols=1)
plt.xlim(-600, 0)
plt.ylim(12, 30)
plt.axvspan(-330, -260, alpha=0.25)
plt.axvspan(-39, 0, alpha=0.25)
plt.grid()
plt.xlabel('Time (Ma)')
plt.ylabel('Temp (°C)')

# plt.subplot(212)
for ind, sorted_ind in enumerate(sorted_inds):
    plt.subplot(212)
    plt.plot(times[:61]*-1, nu_total_arc_contributions[sorted_ind],
             color=colors[ind], lw=2)
plt.xlim(-600, 0)
plt.semilogy()
plt.axvspan(-330, -260, alpha=0.25)
plt.axvspan(-39, 0, alpha=0.25)
plt.grid()
plt.xlabel('Time (Ma)')
plt.ylabel('CW flux (t/a)')
plt.tight_layout()

In [ ]:
# nu uses the previous time step as well
arc_cw_fluxes = np.array([40, 150, 400])  # t/km2/a gaillardet ++ 2011
arc_widths = np.array([10, 25, 50])
arc_fractions = np.array([0.1, 0.25, 0.5])
input_variables = []
extra_nu_temp_results = []
extra_nu_time_results = []
extra_nu_arc_contributions_per_km2 = []
extra_nu_total_arc_contributions = []
for ind, arc_cw_flux in enumerate(arc_cw_fluxes):
    for ind2, width in enumerate(arc_widths):
        for ind3, fractions in enumerate(arc_fractions):

            tropical_arc_weathering = length_of_arcs_in_tropics * \
                width * fractions * arc_cw_flux
            # add previous step to current step
            nu_weather = []
            for ind, i in enumerate(tropical_arc_weathering):
                try:
                    nu_weather.append(i + tropical_arc_weathering[ind+1])
                except:
                    nu_weather.append(i)
            nu_weather = np.asarray(nu_weather)

            # what does this mean geologically in terms of missing arcs?
            nu_weather[45:47] = nu_weather[45:47] + 3e8

            extra_nu_arc_contributions_per_km2.append(
                width * fractions * arc_cw_flux)
            extra_nu_total_arc_contributions.append(nu_weather)

            res, interpstack, lipstack = pySCION_initialise.pySCION_initialise(
                -1, nu_weather)

            extra_nu_temp_results.append(res.state.tempC)
            extra_nu_time_results.append(res.state.time_myr)
            input_variables.append((arc_cw_flux, width, fractions))

In [ ]:
# sorted_arc_contribution, sorted_temp_results, sorted_time_results, sorted_inputs = zip(
#    *sorted(zip(arc_contribution, temp_results, time_results, inputs)))

extra_nu_temp_results = np.asarray(extra_nu_temp_results, dtype=object)
extra_nu_time_results = np.asarray(extra_nu_time_results, dtype=object)
extra_nu_arc_contributions_per_km2 = np.asarray(extra_nu_arc_contributions_per_km2)
extra_nu_total_arc_contributions = np.asarray(extra_nu_total_arc_contributions)
input_variables = np.asarray(input_variables)


colors = cm.berlin(np.linspace(0, 1, len(extra_nu_arc_contributions_per_km2)))
sorted_inds = np.argsort(extra_nu_arc_contributions_per_km2)
plt.figure(figsize=(18, 15))
for ind, sorted_ind in enumerate(sorted_inds):
    default_result = False
    label = f'weather flux {input_variables[sorted_ind][0]} || arc fraction {input_variables[sorted_ind][2]} || arc width {input_variables[sorted_ind][1]}'
    plt.subplot(211)
    plt.plot(extra_nu_time_results[sorted_ind], extra_nu_temp_results[sorted_ind],
             color=colors[ind], label=label, lw=2)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncols=1)
plt.xlim(-600, 0)
plt.ylim(12, 30)
plt.axvspan(-330, -260, alpha=0.25)
plt.axvspan(-39, 0, alpha=0.25)
plt.grid()
plt.xlabel('Time (Ma)')
plt.ylabel('Temp (°C)')

# plt.subplot(212)
for ind, sorted_ind in enumerate(sorted_inds):
    plt.subplot(212)
    plt.plot(times[:61]*-1, extra_nu_total_arc_contributions[sorted_ind],
             color=colors[ind], lw=2)
plt.xlim(-600, 0)
plt.semilogy()
plt.axvspan(-330, -260, alpha=0.25)
plt.axvspan(-39, 0, alpha=0.25)
plt.grid()
plt.xlabel('Time (Ma)')
plt.ylabel('CW flux (t/a)')
plt.tight_layout()

In [29]:
extra_weathering = 400 # t/km2/a gaillardet ++ 2011
arc_width = 50
area_of_arcs_in_tropics = length_of_arcs_in_tropics * arc_width * 0.25 # 0.25 fraction of exposed island arc along subduction strile
tropical_arc_weathering = area_of_arcs_in_tropics * extra_weathering
default_arc_weathering = np.zeros_like(tropical_arc_weathering)

arc_interp = interp1d(times[:61], tropical_arc_weathering)

In [30]:

# why is the orange spike so big?
nu_weather = []
for ind, i in enumerate(tropical_arc_weathering):
    try:
        nu_weather.append(i + tropical_arc_weathering[ind+1])
    except:
        nu_weather.append(i)

nu_weather = np.asarray(nu_weather)

In [ ]:
nu_length_of_arcs_in_tropics = np.copy(length_of_arcs_in_tropics)
nu_length_of_arcs_in_tropics[43:47] = nu_length_of_arcs_in_tropics[43:47] * 2

plt.plot(length_of_arcs_in_tropics)
plt.plot(nu_length_of_arcs_in_tropics)